### Topic Modeling with Latent Dirichlet Allocation (LDA)

In this notebook we use Latent Dirichlet Allocation (LDA) to look for topics in the ride reviews. It is based on material supplied by Cloudera under their Cloudera Academic Partner program and *Spark: The Definitive Guide* book by Bill Chambers and Matei Zaharia. 

Topics
- Extracting and transforming features
- Counting frequency of words
- Specifying and fitting a topic model using LDA
- Exploring topics identified by LDA
- Specifying and fitting a topic model using LDA

In [0]:
# Read the ride review data
reviews = spark.read.parquet("/mnt/cis442f-data/duocar/clean/ride_reviews/")
reviews.head(5) 

Out[1]: [Row(ride_id='0000000009', review='Dale is extremely cordial.'),
 Row(ride_id='0000000037', review='Very junky car.'),
 Row(ride_id='0000000071', review='most awful stench of all time! throw away your air freshener!'),
 Row(ride_id='0000000083', review='No trouble of note.'),
 Row(ride_id='0000000086', review='The driver drove so well!!')]

#### Extracting and transforming features

The ride reviews are not in a form amenable to machine learning algorithms. Spark MLlib provides a number of feature extractors and feature transformers to preprocess the ride reviews into a form appropriate for modeling.


**Parse the ride reviews**
- Use the [RegexTokenizer](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.RegexTokenizer.html) class to improve the tokenization
- Use the [StopWordsRemover](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StopWordsRemover.html) class to remove common words

In [0]:
# Use RegexTokenizer class to tokenize reviews (configured to remove punctuation)
from pyspark.ml.feature import RegexTokenizer
tokenizer = RegexTokenizer(inputCol="review", outputCol="words", gaps=False, pattern="[a-zA-z-']+")
tokenized = tokenizer.transform(reviews)

In [0]:
# Use StopWordsRemover to remove common words
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="words_removed")
remover.getStopWords()[:10]
removed = remover.transform(tokenized)
for row in removed.head(5):
    print(row)

Row(ride_id='0000000009', review='Dale is extremely cordial.', words=['dale', 'is', 'extremely', 'cordial'], words_removed=['dale', 'extremely', 'cordial'])
Row(ride_id='0000000037', review='Very junky car.', words=['very', 'junky', 'car'], words_removed=['junky', 'car'])
Row(ride_id='0000000071', review='most awful stench of all time! throw away your air freshener!', words=['most', 'awful', 'stench', 'of', 'all', 'time', 'throw', 'away', 'your', 'air', 'freshener'], words_removed=['awful', 'stench', 'time', 'throw', 'away', 'air', 'freshener'])
Row(ride_id='0000000083', review='No trouble of note.', words=['no', 'trouble', 'of', 'note'], words_removed=['trouble', 'note'])
Row(ride_id='0000000086', review='The driver drove so well!!', words=['the', 'driver', 'drove', 'so', 'well'], words_removed=['driver', 'drove', 'well'])

#### Count the frequency of words in each review

In [0]:
# Count the words
from pyspark.ml.feature import CountVectorizer
vectorizer = CountVectorizer(inputCol="words_removed", outputCol="words_vectorized", vocabSize=100)
vectorizer_model = vectorizer.fit(removed)
vectorized = vectorizer_model.transform(removed)
# vectorized.head(5)

In [0]:
# Our vocabulary seems reasonable now
print(list(enumerate(vectorizer_model.vocabulary[:100])))

[(0, 'ride'), (1, 'driver'), (2, 'air'), (3, 'freshener'), (4, 'car'), (5, 'vehicle'), (6, 'really'), (7, 'time'), (8, 'due'), (9, 'experience'), (10, 'happy'), (11, 'service'), (12, 'pleased'), (13, 'fine'), (14, "driver's"), (15, 'appreciate'), (16, 'bad'), (17, 'awful'), (18, 'stunk'), (19, 'conversation'), (20, 'smelled'), (21, 'drives'), (22, 'super'), (23, 'extremely'), (24, 'reeked'), (25, 'felt'), (26, 'polite'), (27, 'radio'), (28, 'clean'), (29, 'nice'), (30, 'horrible'), (31, 'marvelous'), (32, 'refused'), (33, 'windows'), (34, 'breathe'), (35, 'fresheners'), (36, 'made'), (37, 'wanted'), (38, 'nicely'), (39, 'terrible'), (40, 'drove'), (41, 'music'), (42, 'good'), (43, 'unpleasant'), (44, 'talk'), (45, 'loud'), (46, 'tidy'), (47, 'talking'), (48, 'full'), (49, 'constantly'), (50, 'stereo'), (51, 'whole'), (52, 'superb'), (53, 'great'), (54, 'swell'), (55, 'entire'), (56, 'ok'), (57, 'want'), (58, 'uncomfortable'), (59, 'excellent'), (60, 'get'), (61, 'exceptional'), (62, 'considerate'), (63, 'annoying'), (64, 'perfectly'), (65, 'gave'), (66, 'chat'), (67, 'chatting'), (68, 'forever'), (69, 'speak'), (70, 'disrespectful'), (71, 'kept'), (72, 'pleasant'), (73, 'terrific'), (74, 'kind'), (75, 'courteous'), (76, 'enough'), (77, 'inconsiderate'), (78, 'thank'), (79, 'decent'), (80, 'chatted'), (81, 'speaking'), (82, 'acceptable'), (83, 'irritating'), (84, 'quit'), (85, 'rude'), (86, 'still'), (87, 'cease'), (88, 'give'), (89, 'totally'), (90, 'stop'), (91, 'fully'), (92, 'trying'), (93, 'much'), (94, 'without'), (95, 'stopping'), (96, 'well'), (97, 'stink'), (98, 'thanks'), (99, 'turn')]

In [0]:
# We can see how the processed reviews are vectorized
vectorized.select("words_removed", "words_vectorized").head(5)

Out[6]: [Row(words_removed=['dale', 'extremely', 'cordial'], words_vectorized=SparseVector(100, {23: 1.0})),
 Row(words_removed=['junky', 'car'], words_vectorized=SparseVector(100, {4: 1.0})),
 Row(words_removed=['awful', 'stench', 'time', 'throw', 'away', 'air', 'freshener'], words_vectorized=SparseVector(100, {2: 1.0, 3: 1.0, 7: 1.0, 17: 1.0})),
 Row(words_removed=['trouble', 'note'], words_vectorized=SparseVector(100, {})),
 Row(words_removed=['driver', 'drove', 'well'], words_vectorized=SparseVector(100, {1: 1.0, 40: 1.0, 96: 1.0}))]

#### Specify and fit a topic model using LDA

In [0]:
# Use the `LDA` class to specify an LDA model
from pyspark.ml.clustering import LDA
lda = LDA(featuresCol="words_vectorized", k=9)

# Use the `fit` method to fit the LDA model
lda_model = lda.fit(vectorized)

#### Explore the topics identified by LDA

In [0]:
# lda_model.describeTopics().show(truncate=False)
lda_model.describeTopics().show()

+-----+--------------------+--------------------+
topic| termIndices| termWeights|
+-----+--------------------+--------------------+
 0|[90, 49, 11, 72, ...|[0.01233804038514...|
 1|[11, 59, 52, 9, 4...|[0.12427603722302...|
 2|[27, 45, 93, 41, ...|[0.14359815282664...|
 3|[2, 3, 4, 8, 14, ...|[0.16685897318617...|
 4|[50, 26, 1, 29, 9...|[0.09192115679844...|
 5|[33, 1, 71, 58, 3...|[0.11439660147973...|
 6|[0, 1, 15, 10, 12...|[0.31353005697997...|
 7|[1, 19, 49, 37, 3...|[0.16619760156807...|
 8|[1, 5, 6, 4, 9, 2...|[0.16394327857149...|
+-----+--------------------+--------------------+

In [0]:
topics = lda_model.topicsMatrix()
vocab = vectorizer_model.vocabulary
wordNumbers = 10  # number of words per topic

# Capture the data from the model's describeTopics() method
topics_data = (lda_model.describeTopics(maxTermsPerTopic = wordNumbers)).collect()

# Extract the vocabulary associated with each term associated with each topic
topics = []
terms = []
terms_in_words = []
weights = []

for row in topics_data:
    words =[]
    topics.append(row[0])
    terms.append(row[1])
    for term in (row[1]):
        word = vocab[term]
        words.append(word)
    terms_in_words.append(words)    
    weights.append(row[2])

# Print vocuabulary for each topic discoverd by LDA
for topic in topics:
    print (("Topic: " + str(topics[topic])), end="  ")
    print (terms_in_words[topic])
    # print (weights[topic])

# Note: If we were working with this sort of data a lot it would make sense to turn this sort of analysis into a function that we could call at will

Topic: 0 ['stop', 'constantly', 'service', 'pleasant', 'driver', 'superb', 'freshener', 'rude', 'want', 'nicely']
Topic: 1 ['service', 'excellent', 'superb', 'experience', 'good', 'thanks', 'great', 'thank', 'swell', 'fine']
Topic: 2 ['radio', 'loud', 'much', 'music', 'turn', "driver's", 'inconsiderate', 'drove', 'bad', 'awful']
Topic: 3 ['air', 'freshener', 'car', 'due', "driver's", 'vehicle', 'bad', 'ride', 'time', 'reeked']
Topic: 4 ['stereo', 'polite', 'driver', 'nice', 'fully', 'pleased', 'chat', 'loud', 'awful', 'good']
Topic: 5 ['windows', 'driver', 'kept', 'uncomfortable', 'refused', 'rude', 'inconsiderate', 'experience', 'ride', 'full']
Topic: 6 ['ride', 'driver', 'appreciate', 'happy', 'pleased', 'time', 'fine', 'service', 'acceptable', 'perfectly']
Topic: 7 ['driver', 'conversation', 'constantly', 'wanted', 'refused', 'annoying', 'trying', 'want', 'quit', 'talking']
Topic: 8 ['driver', 'vehicle', 'really', 'car', 'experience', 'drives', 'tidy', 'clean', 'nicely', 'enough']

#### Apply the topic model

In [0]:
reviews_with_topics = lda_model.transform(vectorized)
for row in reviews_with_topics.select("review", "topicDistribution").head(15):
    print (row)

Row(review='Dale is extremely cordial.', topicDistribution=DenseVector([0.0503, 0.0511, 0.0513, 0.0591, 0.5608, 0.0511, 0.065, 0.0535, 0.058]))
Row(review='Very junky car.', topicDistribution=DenseVector([0.0503, 0.0511, 0.0513, 0.5673, 0.0524, 0.0511, 0.065, 0.0534, 0.058]))
Row(review='most awful stench of all time! throw away your air freshener!', topicDistribution=DenseVector([0.0199, 0.0202, 0.0203, 0.8287, 0.0208, 0.0202, 0.0258, 0.0212, 0.023]))
Row(review='No trouble of note.', topicDistribution=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))
Row(review='The driver drove so well!!', topicDistribution=DenseVector([0.0249, 0.0253, 0.0254, 0.0293, 0.026, 0.0254, 0.0322, 0.0265, 0.785]))
Row(review='really irritating driver refused to cease chatting!!', topicDistribution=DenseVector([0.0142, 0.0144, 0.0145, 0.0167, 0.0148, 0.0144, 0.0184, 0.8763, 0.0164]))
Row(review='I could not breathe due to the air freshener.', topicDistribution=DenseVector([0.0199, 0.0202, 0.0203, 0.8287, 0.0208, 0.0202, 0.0257, 0.0212, 0.023]))
Row(review='just adequate', topicDistribution=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))
Row(review='the driver drove so nicely!!', topicDistribution=DenseVector([0.0249, 0.0253, 0.0254, 0.0293, 0.026, 0.0254, 0.0322, 0.0265, 0.785]))
Row(review='Pleased with this ride.', topicDistribution=DenseVector([0.0333, 0.0339, 0.034, 0.0391, 0.0348, 0.0339, 0.7172, 0.0354, 0.0384]))
Row(review='The stereo was blaring', topicDistribution=DenseVector([0.0503, 0.0511, 0.0513, 0.059, 0.5608, 0.0511, 0.065, 0.0534, 0.058]))
Row(review='I made it there as expected', topicDistribution=DenseVector([0.0503, 0.0511, 0.0513, 0.0591, 0.0524, 0.0511, 0.5733, 0.0534, 0.058]))
Row(review='this driver was not polite and wanted to talk on and on', topicDistribution=DenseVector([0.0199, 0.0202, 0.0203, 0.0234, 0.2582, 0.0202, 0.0258, 0.589, 0.023]))
Row(review='It was a delightful ride', topicDistribution=DenseVector([0.0503, 0.0511, 0.0513, 0.059, 0.0524, 0.0511, 0.5734, 0.0534, 0.058]))
Row(review='The driver drove super well.', topicDistribution=DenseVector([0.0199, 0.0202, 0.0203, 0.0234, 0.0208, 0.0202, 0.0257, 0.0212, 0.8283]))

###Hands On

![Hands-on](https://cis442f-open-data.s3.amazonaws.com/pictures/hands.png "Hands-on")


#### Exercises

(1) Determine if increasing the vocabulary size improves the solution.

(2) Experiment with different hyperparameters for the LDA algorithm.

(3) Use the `HashingTF` estimator rather than the `CountVectorizer` estimator to generate the term-frequency vectors.

(4) Use the `NGram` transformer to consider pairs of words rather than single words.



**References**

[Wikipedia - Latent Dirichlet allocation](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation)

[Spark Documentation - Latent Dirichlet allocation](http://spark.apache.org/docs/latest/ml-clustering.html#latent-dirichlet-allocation-lda)

[Spark Python API - LDA, LDAModel, localLDAModel, and DistributedLDAModel classes](https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html#clustering)